# Processing of MalariaMolecules.csv
Processing of original data from [MasterList](https://docs.google.com/spreadsheets/d/1Rvy6OiM291d1GN_cyT6eSw_C3lSuJ1jaR7AJa8hgGsc/edit#gid=510297618) containing all available information for OSM compounds (from different series).
Downloaded on 20.05.21

In [2]:
import pandas as pd
import numpy as np
import csv
import os
from scipy.stats import rankdata
from rdkit import Chem
import rdkit.Chem.rdFMCS as rdFMCS
PATH="../data/OriginalData"

## Select Activity values 
We only keep activity data for inherited EC50 and experimental IC50s (not including experiments against multidrug resistant strains Dd2 and K1

In [3]:
#open MalariaMolecules as a dataframe taking only columns of interest
df=pd.read_csv(os.path.join(PATH, "MalariaMolecules.csv"), usecols = ["OSM Number", "SMILES", "Series", "PfaI EC50 (Inh)", "Pfal IC50 (GSK)", "Pfal IC50 (Syngene)","Pfal IC50 (Dundee)", "Pfal IC50 (Avery)", "Pfal IC50 (Ralph)", "Pfal IC50 (Guy)", "Pfal IC50 (Batra)", "Pfal (3D7) IC50 (Broad)"])
#rename the columns for more consistent nomenclature
df.rename(columns={"OSM Number":"OSMNumber",
                   "SMILES":"smiles",
                   "Series":"series",
                   "PfaI EC50 (Inh)":"EC50_Inh", 
                   "Pfal IC50 (GSK)":"IC50_GSK", 
                   "Pfal IC50 (Syngene)":"IC50_Syngene",
                   "Pfal IC50 (Dundee)":"IC50_Dundee", 
                   "Pfal IC50 (Avery)":"IC50_Avery", 
                   "Pfal IC50 (Ralph)":"IC50_Ralph", 
                   "Pfal IC50 (Guy)":"IC50_Guy",
                   "Pfal IC50 (Batra)":"IC50_Batra", 
                   "Pfal (3D7) IC50 (Broad)":"IC50_Broad"},
                   inplace=True)

## Clean data for average activity 
Dundee, Avery, GSK and Syngene IC50 columns contain several values separated by , or ; which must be added in different columns for average calculation

In [4]:
#Delete Qualifier > for easy averaging
df=df.replace({'>':''}, regex=True)

#eliminate rows where data is not given as IC50 but as % of inhibition at certain uM concentration.
#these rows all relate to IC50_Avery and series 1 or 3 compounds
df2=df[df['IC50_Avery'].str.contains('[A-Za-z]', na=False)]
df3=df.drop(df[df['IC50_Avery'].str.contains('[A-Za-z]', na=False)].index)
df3.to_csv(os.path.join(PATH,"all_series_notsep.csv"), index=False)

#split columns that have 2 or more values as different experiments (some are separated by "," and some by ";")
df=pd.read_csv(os.path.join(PATH,"all_series_notsep.csv"))
#create a dataframe for each splitted column
df4=pd.DataFrame(df.IC50_Dundee.str.split(",",expand=True).values.tolist(), columns=["IC50_Dundee1","IC50_Dundee2","IC50_Dundee3","IC50_Dundee4","IC50_Dundee5"])
df5=pd.DataFrame(df.IC50_Syngene.str.split(",",expand=True).values.tolist(), columns=["IC50_Syngene1","IC50_Syngene2"])
df6=pd.DataFrame(df.IC50_GSK.str.split(";",expand=True).values.tolist(), columns=["IC50_GSK1","IC50_GSK2"])
df7=pd.DataFrame(df.IC50_Avery.str.split(";",expand=True).values.tolist(), columns=["IC50_Avery1","IC50_Avery2"])
dataframes=[df4,df5, df6, df7]
#add the dataframes to the full dataframe
df8=df.join(dataframes, how="outer")
df8= df8.fillna(value=np.nan)
#eliminate the columns containing multiple values
df9=df8.drop(columns = ["IC50_Dundee", "IC50_Syngene", "IC50_GSK", "IC50_Avery"])

### Calculate mean activity
Include all IC50 as individual values

In [5]:
#calculate mean activity in new column
cols = ["EC50_Inh", 
          "IC50_GSK1",
          "IC50_GSK2",
          "IC50_Syngene1", 
          "IC50_Syngene2",
          "IC50_Dundee1", 
          "IC50_Dundee2",
          "IC50_Dundee3", 
          "IC50_Dundee4", 
          "IC50_Dundee5",
          "IC50_Avery1",
          "IC50_Avery2",
          "IC50_Ralph", 
          "IC50_Guy",
          "IC50_Batra", 
          "IC50_Broad"]
df9[cols] = df9[cols].apply(pd.to_numeric, errors='raise', axis=1)

df9['activity'] = df9[["EC50_Inh", 
                          "IC50_GSK1",
                          "IC50_GSK2",
                          "IC50_Syngene1", 
                          "IC50_Syngene2",
                          "IC50_Dundee1", 
                          "IC50_Dundee2",
                          "IC50_Dundee3", 
                          "IC50_Dundee4", 
                          "IC50_Dundee5",
                          "IC50_Avery1",
                          "IC50_Avery2",
                          "IC50_Ralph", 
                          "IC50_Guy",
                          "IC50_Batra", 
                          "IC50_Broad"]].mean(axis=1, numeric_only=None)
#drop all columns except mean
df10=df9.drop(columns = ["EC50_Inh", 
                          "IC50_GSK1",
                          "IC50_GSK2",
                          "IC50_Syngene1", 
                          "IC50_Syngene2",
                          "IC50_Dundee1", 
                          "IC50_Dundee2",
                          "IC50_Dundee3", 
                          "IC50_Dundee4", 
                          "IC50_Dundee5",
                          "IC50_Avery1",
                          "IC50_Avery2",
                          "IC50_Ralph", 
                          "IC50_Guy",
                          "IC50_Batra", 
                          "IC50_Broad"])
#delete rows without smiles
df11=df10.dropna(subset=["smiles"])

### Change to canonical smiles
Change Smiles strings to canonical smiles using canonical_smiles.py script

In [6]:
#add canonical smiles
from scripts.canonical_smiles import read_smiles_file #python script from Reinvent modified so if canonical smiles is None, keep original smiles

df12=df11["smiles"]
df12.to_csv(os.path.join(PATH,"smiles.csv"), index=False, header=False)

canonicals = []
for smiles in read_smiles_file(os.path.join(PATH,"smiles.csv"), ignore_invalid=False):
    canonicals += [smiles]

df11["canonical"] = canonicals
df11=df11.drop(columns=["smiles"])

#keep .csv with all series
df11.to_csv(os.path.join(PATH,"all_series_IC50mean.csv"), index=False)


C:\Users\Gemma\Anaconda3\envs\malariaOSM\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


## Keep only series 4 compounds for subsequent analysis
Includes the test set of competition Round 2 and the new candidates proposed.
OSM-LO-48 and OSM-LO-49 are new candidates from the ChemArxv series4-competition paper for which experimental data is available but not entered in the Master List.

In [7]:
df=pd.read_csv(os.path.join(PATH, "all_series_IC50mean.csv"))

#select only series 4
df2=df.loc[df["series"]=="4"]

#find test molecules without IC50 and add them from ChemArxv paper
exscientia2=df2.loc[df2["OSMNumber"]=="OSM-LO-48"].index[0]
molomics2=df2.loc[df2["OSMNumber"]=="OSM-LO-49"].index[0]

df2.loc[exscientia2,"activity"]=2.42
df2.loc[molomics2,"activity"]=1.24

df2.to_csv(os.path.join(PATH,"series4_IC50mean.csv"), index=False)

C:\Users\Gemma\Anaconda3\envs\malariaOSM\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


### Clean Series 4 Dataset
Some series 4 compounds do not maintain the triazolopyrazine core, and are therefore discarded to avoid further diversification of the core in subsequent steps.
Compounds without experimental activity available and no smiles (or non canonical smiles) are also eliminated. 
Finally, activity output is classified as active (0-2.5 uM) or inactive (>2.5 uM), according to [OSM classification](https://github.com/OpenSourceMalaria/Series4_PredictiveModel/issues/1#issuecomment-524913974) to obtain binary activity values (0: inactive - 1: active). A regression with activity values normalized from 0 to 1 is also added to the final file.

In [10]:
# 1. Remove molecules without the triazolopyrazine core using the Maximmum Common Substructre (MCS) from rdKit
    # SMILES: C1=NN=C2C=NC=CN12 --> if not 9 matches count as not series4

df = pd.read_csv(os.path.join(PATH,"series4_IC50mean.csv"))
series4 = []
osms_ = list(df["OSMNumber"])
osms = []
for i, m in enumerate(list(df["canonical"])):
    mol = Chem.MolFromSmiles(m)
    if mol is None: continue
    series4 += [m]
    osms += [osms_[i]]
refmol = Chem.MolFromSmiles("C1=NN=C2C=NC=CN12")
R = []
for i, m in enumerate(series4):
    mol = Chem.MolFromSmiles(m)
    mcs = rdFMCS.FindMCS([mol, refmol])
    if mcs.numAtoms != 9:
        R += [[osms[i], m, mcs.numAtoms]]

#create a dataframe with molecules that are not actually series4
df_nots4 = pd.DataFrame(R, columns=["OSMNumber", "canonical", "mcs"])

#eliminate not series 4 from main data
df2=pd.merge(df,df_nots4, how="outer", indicator=True) #join dataframes adding _merge column ("both" if smiles coincide)
df3=df2[df2._merge == "both"]
df4=df2[df2._merge != "both"] #eliminate rows that are identified as both
df4=df4.drop(columns = ["_merge", "mcs"])

#check we have eliminated all non_s4 molecules
def moleculenumber(df1, df2, df3):
    if len((df3.index)) == len((df1.index))-len((df2.index)):
        return True
    else:
        return False
        
print(moleculenumber(df, df_nots4, df4))

# 2. Eliminate incorrect SMILES

idxs=[]
for i, smi in enumerate (list(df4["canonical"])):
    if Chem.MolFromSmiles(smi):
        idxs += [i]       
df=df4.iloc[idxs]

#keep all s4 correct smiles in a list to check newly generated molecules are not already existing ones
series4_smi=df["canonical"] 
series4_smi.to_csv(os.path.join(PATH,"series4_allsmiles.csv"), index=False)

# 4. Eliminate molecules without activity
df=df[df["activity"].notnull()]

# 3. Process activity data for:
    #Regression (normalized 0 to 1)
    #Classification (0 or 1)
    
#normalize activity (0 to 1) with rank function
y = np.array(df["activity"])
r = rankdata (-y, method="ordinal")/len(y) #order activities from 0 to inf and divide each position by total number (1=higher activity, 0=low activity)
#create binary code for activity (0=inactive IC50>2.5, 1=active, IC50<=2.5)
b = np.zeros(len(y))
b[y<=2.5]=1
#add to dataframe
df=pd.DataFrame({
    "osm": list(df["OSMNumber"]),
    "smiles": list(df["canonical"]),
    "activity": list(df["activity"]),
    "norm_activity": r,
    "bin_activity": b
})

df.to_csv(os.path.join(PATH,"series4_processed.csv"), index=False)


True


#### Separate the 2019 test set for autoML validation
AutoML activity predictor has been validated emulatins the competition round 2. The model predicted the actvity for the 32 test molecules with an 80% accuracy.

In [11]:
df=pd.read_csv(os.path.join(PATH,"series4_processed.csv"))
df_test=pd.read_csv(os.path.join(PATH,"test_set.csv"))

df_train = df[~df['osm'].isin(df_test['osm'])] #eliminate test molecules

#format for autoML

df_train_clf=df_train.loc[:,["smiles","bin_activity"]]
df_train_reg=df_train.loc[:,["smiles","norm_activity"]]

df_train_clf.to_csv(os.path.join(PATH, "series4_train_clf.csv"), index=False)
df_train_reg.to_csv(os.path.join(PATH,"series4_train_reg.csv"), index=False)